In [104]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (183 kB/s)
Reading package lists... Done


In [105]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [106]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

df.show()

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [107]:
# ensure correct format for each column 
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [108]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("homeSale")

In [109]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT EXTRACT(YEAR FROM date) AS year, bedrooms, ROUND(AVG(price), 2) AS average_price \
          FROM homeSale \
          WHERE bedrooms = 4 \
          GROUP BY year, bedrooms \
          ORDER BY year").show()

+----+--------+-------------+
|year|bedrooms|average_price|
+----+--------+-------------+
|2019|       4|     300263.7|
|2020|       4|    298353.78|
|2021|       4|    301819.44|
|2022|       4|    296363.88|
+----+--------+-------------+



In [110]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT EXTRACT(YEAR FROM date) AS year, bedrooms, bathrooms, ROUND(AVG(price), 2) AS average_price \
          FROM homeSale \
          WHERE bedrooms = 3 AND bathrooms = 3\
          GROUP BY year, bedrooms, bathrooms \
          ORDER BY year").show()


+----+--------+---------+-------------+
|year|bedrooms|bathrooms|average_price|
+----+--------+---------+-------------+
|2019|       3|        3|    287287.82|
|2020|       3|        3|    294204.16|
|2021|       3|        3|    294211.46|
|2022|       3|        3|    292725.69|
+----+--------+---------+-------------+



In [116]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("SELECT EXTRACT(YEAR FROM date) AS year, bedrooms, bathrooms, floors, ROUND(AVG(price), 2) AS average_price \
          FROM homeSale \
          WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000\
          GROUP BY year, bedrooms, bathrooms, floors\
          ORDER BY year").show()

+----+--------+---------+------+-------------+
|year|bedrooms|bathrooms|floors|average_price|
+----+--------+---------+------+-------------+
|2019|       3|        3|     2|    289859.14|
|2020|       3|        3|     2|    292289.09|
|2021|       3|        3|     2|    296330.96|
|2022|       3|        3|     2|    290242.99|
+----+--------+---------+------+-------------+



In [117]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("SELECT ROUND(avg(view), 2) AS average_view_rating FROM homeSale WHERE price >= 350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.19367218017578125 seconds ---


In [118]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homeSale")

DataFrame[]

In [119]:
# 8. Check if the table is cached.
spark.catalog.isCached("homeSale")

True

In [120]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT ROUND(avg(view), 2) AS average_view_rating FROM homeSale WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.17564749717712402 seconds ---


In [121]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("homeSale_partitioned")
partition_df = spark.read.parquet('homeSale_partitioned')

In [123]:
# 11. Read the parquet formatted data.
partition_df.show()

+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|               date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06 00:00:00|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09 00:00:00|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08 00:00:00|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04 00:00:00|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17 00:00:00|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a

In [124]:
# 12. Create a temporary table for the parquet data.
partition_df.createOrReplaceTempView("homeSale_partition")

In [125]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT ROUND(avg(view), 2) AS average_view_rating FROM homeSale_partition WHERE price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|average_view_rating|
+-------------------+
|              32.26|
+-------------------+

--- 0.7500169277191162 seconds ---


In [126]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homeSale")

DataFrame[]

In [127]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("homeSale")

False